# GW-EM Association Analysis Framework

## Introduction

This notebook demonstrates how to use the **GW-EM (Gravitational Wave - Electromagnetic) Association Framework** to evaluate whether an electromagnetic transient (like a kilonova, GRB, or afterglow) is associated with a gravitational wave event using Bayesian statistics.

### What This Framework Does

The framework computes the **posterior probability** that an EM transient is the counterpart to a GW event by evaluating three key overlap integrals:

1. **Spatial Overlap (I_Ω)**: How well does the EM transient position match the GW sky localization?
2. **Distance Overlap (I_DL)**: How well does the EM transient distance (from redshift) match the GW distance posterior?
3. **Temporal Overlap (I_t)**: How well does the EM detection time match the expected time delay for the EM model?

### Bayesian Framework

The framework implements:

$$\text{Posterior Odds} = \text{Prior Odds} \times \text{Bayes Factor}$$

$$\text{Bayes Factor} = \frac{I_Ω \times I_{DL} \times I_t}{P_{\text{chance}}}$$

$$P(\text{Associated}) = \frac{\text{Posterior Odds}}{1 + \text{Posterior Odds}}$$

Where:
- **Prior Odds**: Depends on GW source type (BNS=1.0, NSBH=0.1, BBH=0.01)
- **P_chance**: Chance coincidence rate (expected false alarms)

### Key Features
- **Bayesian Association Analysis**: Compute posterior odds for GW-EM associations
- **Multiple Overlap Integrals**: Spatial (I_Ω), distance (I_DL), and temporal (I_t)
- **EM Models**: Support for kilonova, GRB, and afterglow light curve models
- **Candidate Ranking**: Rank multiple EM candidates by association probability
- **Publication-Quality Plots**: Generate figures for papers and presentations
- **3D Skymap Support**: Handles both 2D and 3D GW skymaps with distance information


In [2]:
import sys
from pathlib import Path

package_path = Path("/Users/kaitlynpak/Desktop/gw_package/gwPackage/src")
print("Package path exists:", package_path.exists())
print("gw_assoc exists:", (package_path / "gw_assoc").exists())

if package_path.exists() and (package_path / "gw_assoc").exists():
    sys.path.insert(0, str(package_path))
    print("✓ Added to sys.path")
else:
    print("✗ Path not found; check the path string above.")

Package path exists: True
gw_assoc exists: True
✓ Added to sys.path


## Setup

First, let's import the necessary packages and verify the installation.


In [3]:
# Standard library imports
import sys
import os
from pathlib import Path
import json

# Scientific computing
import numpy as np
import matplotlib.pyplot as plt

# Astronomy packages
from astropy.cosmology import Planck15
import astropy.units as u

# GW-EM Association Framework
from gw_assoc import Association
from gw_assoc.plots import plot_association_summary, plot_candidate_ranking

# Set up plotting style
plt.style.use('default')
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 11

print("✓ All packages imported successfully!")
print(f"✓ Python version: {sys.version.split()[0]}")
print(f"✓ NumPy version: {np.__version__}")


✓ All packages imported successfully!
✓ Python version: 3.13.5
✓ NumPy version: 2.1.3


### Verify Test Data Availability

Let's check if test data is available. If not, we'll provide instructions on how to create it.


In [4]:
# Check for test data
test_skymap_path = Path("test_data/test_skymap.fits")
test_transients_path = Path("test_data/test_transients.json")

if test_skymap_path.exists():
    print(f"✓ Test skymap found: {test_skymap_path}")
    skymap_file = str(test_skymap_path)
else:
    print(f"⚠ Test skymap not found: {test_skymap_path}")
    print("  You can create test data by running: python test_gw_assoc.py")
    print("  Or use your own skymap file path below.")
    skymap_file = None  # You'll need to set this manually

if test_transients_path.exists():
    print(f"✓ Test transients found: {test_transients_path}")
    with open(test_transients_path) as f:
        test_transients = json.load(f)
    print(f"  Found {len(test_transients)} test transients")
else:
    print(f"⚠ Test transients not found: {test_transients_path}")
    test_transients = None


⚠ Test skymap not found: test_data/test_skymap.fits
  You can create test data by running: python test_gw_assoc.py
  Or use your own skymap file path below.
⚠ Test transients not found: test_data/test_transients.json


## Basic Example: Single Transient Analysis

Let's start with a simple example: analyzing a single EM transient candidate to determine if it's associated with a GW event.

### Step 1: Define the Transient

We'll create a transient with:
- **Position**: Right ascension (RA) and declination (Dec) in degrees
- **Redshift**: Optional, but helps with distance overlap calculation
- **Time**: Detection time (GPS seconds or MJD)
- **GW Time**: The GW event time (from GraceDB or skymap header)


In [5]:
# Example transient information
# This transient is positioned near the center of our test skymap
transient_info = {
    "name": "AT2024example",
    "ra": 120.5,           # Right ascension in degrees
    "dec": -29.8,          # Declination in degrees
    "z": 0.05,             # Redshift (optional but recommended)
    "z_err": 0.005,        # Redshift uncertainty
    "time": 1234567891.0,  # Detection time (GPS seconds)
    "gw_time": 1234567880.0,  # GW event time (GPS seconds)
    "magnitude": 18.5,     # Apparent magnitude (optional)
    "filter_band": "r"     # Filter band (optional)
}

print("Transient Information:")
print(f"  Name: {transient_info['name']}")
print(f"  Position: RA = {transient_info['ra']:.2f}°, Dec = {transient_info['dec']:.2f}°")
print(f"  Redshift: z = {transient_info['z']:.4f} ± {transient_info['z_err']:.4f}")
print(f"  Time delay: {(transient_info['time'] - transient_info['gw_time']):.1f} seconds")


Transient Information:
  Name: AT2024example
  Position: RA = 120.50°, Dec = -29.80°
  Redshift: z = 0.0500 ± 0.0050
  Time delay: 11.0 seconds


### Step 2: Create an Association Object

The `Association` class is the main interface to the framework. It takes:
- **gw_file**: Path to the GW skymap FITS file
- **transient_info**: Dictionary with transient information


In [6]:
# Create association object
# Note: If test_skymap_path doesn't exist, replace with your skymap file path
if skymap_file:
    try:
        assoc = Association(
            gw_file=skymap_file,
            transient_info=transient_info
        )
        print("✓ Association object created successfully!")
        print(f"  GW skymap: {skymap_file}")
        print(f"  Transient: {transient_info['name']}")
    except Exception as e:
        print(f"✗ Error creating association: {e}")
        print("\nMake sure:")
        print("  1. The skymap file exists and is readable")
        print("  2. The package is installed: pip install -e .")
        print("  3. You're in the correct directory")
        assoc = None
else:
    print("⚠ Please set skymap_file to your GW skymap path")
    assoc = None


⚠ Please set skymap_file to your GW skymap path


### Step 3: Compute Association Odds

Now we'll compute the association probability using the `compute_odds()` method. This calculates all three overlap integrals and combines them into a posterior probability.

**Parameters:**
- `em_model`: Type of EM counterpart ('kilonova', 'grb', or 'afterglow')
- `prior_odds`: Prior odds ratio (default 1.0 for BNS, 0.1 for NSBH, 0.01 for BBH)
- `chance_coincidence_rate`: Expected rate of chance coincidences (default 1e-4)
- `H0_uncertainty`: Hubble constant uncertainty in km/s/Mpc (default 7.0)


In [7]:
if assoc is not None:
    # Compute association odds with kilonova model
    results = assoc.compute_odds(
        em_model='kilonova',          # EM counterpart model
        prior_odds=1.0,               # BNS prior (1.0 = EM expected)
        chance_coincidence_rate=1e-5, # Expected false alarm rate
        H0_uncertainty=7.0            # Hubble constant uncertainty (km/s/Mpc)
    )
    
    print("=" * 60)
    print("ASSOCIATION ANALYSIS RESULTS")
    print("=" * 60)
    
    print(f"\nTransient: {transient_info['name']}")
    print(f"Position: RA = {transient_info['ra']:.2f}°, Dec = {transient_info['dec']:.2f}°")
    if transient_info.get('z'):
        print(f"Redshift: z = {transient_info['z']:.4f} ± {transient_info['z_err']:.4f}")
    
    print(f"\n{'─' * 60}")
    print("OVERLAP INTEGRALS:")
    print(f"{'─' * 60}")
    print(f"Spatial (I_Ω):   {results['I_omega']:.6e}")
    print(f"Distance (I_DL): {results['I_dl']:.6e}")
    print(f"Temporal (I_t):  {results['I_t']:.6e}")
    
    print(f"\n{'─' * 60}")
    print("STATISTICAL RESULTS:")
    print(f"{'─' * 60}")
    print(f"Bayes Factor:     {results['bayes_factor']:.6e}")
    print(f"Prior Odds:       {results['prior_odds']:.2f}")
    print(f"Posterior Odds:   {results['posterior_odds']:.6e}")
    print(f"Log₁₀ Odds:       {results['log_posterior_odds']:.2f}")
    
    print(f"\n{'─' * 60}")
    print("ASSOCIATION PROBABILITY:")
    print(f"{'─' * 60}")
    prob = results['confidence']
    print(f"P(Associated) = {prob:.1%}")
    
    # Interpretation
    if prob > 0.95:
        decision = "🌟 STRONG ASSOCIATION - Very likely the EM counterpart!"
    elif prob > 0.5:
        decision = "✓ PROBABLE ASSOCIATION - Good candidate for follow-up"
    elif prob > 0.1:
        decision = "⚠ WEAK ASSOCIATION - Possible but unlikely"
    else:
        decision = "✗ NO ASSOCIATION - Very unlikely to be related"
    
    print(f"\nDecision: {decision}")
else:
    print("⚠ Cannot compute odds - association object not created")
    results = None


⚠ Cannot compute odds - association object not created


### Interpreting the Results

**Overlap Integrals:**
- **I_Ω (Spatial)**: Values > 1 indicate the transient is in a high-probability region of the GW skymap. Values < 1 indicate it's in a low-probability region.
- **I_DL (Distance)**: Values > 1 indicate good distance agreement. Values < 1 indicate distance mismatch.
- **I_t (Temporal)**: Values close to 1 indicate the time delay matches the EM model expectations.

**Bayes Factor:**
- BF > 1: Evidence favors association
- BF < 1: Evidence favors no association
- Jeffrey's scale: BF > 10 (strong), BF > 100 (very strong)

**Posterior Probability:**
- P > 0.5: More likely associated than not
- P > 0.95: Strong association (high confidence)
- P < 0.1: Very unlikely to be associated


## Visualization

Let's create visualizations to better understand the association results.

### Skymap Plot

First, let's plot the GW skymap with the transient position marked.


In [8]:
if assoc is not None:
    try:
        # Plot skymap with transient position
        output_file = "skymap_with_transient.png"
        assoc.plot_skymap(output_file)
        print(f"✓ Skymap plot saved to: {output_file}")
        
        # Display the plot inline
        from IPython.display import Image, display
        if Path(output_file).exists():
            display(Image(output_file))
    except Exception as e:
        print(f"⚠ Could not create skymap plot: {e}")
        print("  This may require healpy to be installed: pip install healpy")
else:
    print("⚠ Cannot create plot - association object not created")


⚠ Cannot create plot - association object not created


### Summary Plot

Create a comprehensive summary plot showing all the association statistics.


In [9]:
if results is not None:
    try:
        # Create summary plot
        summary_file = "association_summary.png"
        plot_association_summary(results, summary_file)
        print(f"✓ Summary plot saved to: {summary_file}")
        
        # Display the plot inline
        from IPython.display import Image, display
        if Path(summary_file).exists():
            display(Image(summary_file))
    except Exception as e:
        print(f"⚠ Could not create summary plot: {e}")
        import traceback
        traceback.print_exc()
else:
    print("⚠ Cannot create plot - no results available")


⚠ Cannot create plot - no results available


## Multiple Candidates Ranking

In real GW follow-up campaigns, you often have multiple EM transient candidates. The framework can rank them by association probability to help prioritize follow-up observations.

### Load Multiple Candidates

Let's load multiple candidates and rank them.


In [10]:
# Define multiple candidate transients
candidates = [
    {
        "name": "AT2024best",
        "ra": 120.2,      # Close to skymap center
        "dec": -30.1,
        "z": 0.049,
        "z_err": 0.003,
        "time": 1234567890.0,
        "gw_time": 1234567880.0,
        "magnitude": 17.5
    },
    {
        "name": "AT2024okay",
        "ra": 121.5,      # Slightly offset
        "dec": -29.0,
        "z": 0.052,
        "z_err": 0.005,
        "time": 1234567900.0,
        "gw_time": 1234567880.0,
        "magnitude": 18.0
    },
    {
        "name": "AT2024poor",
        "ra": 125.0,      # Further from center
        "dec": -35.0,
        "z": 0.045,
        "z_err": 0.004,
        "time": 1234568000.0,
        "gw_time": 1234567880.0,
        "magnitude": 18.5
    },
    {
        "name": "AT2024bad",
        "ra": 180.0,      # Far from localization
        "dec": 0.0,
        "z": 0.1,
        "z_err": 0.01,
        "time": 1234569000.0,
        "gw_time": 1234567880.0,
        "magnitude": 19.0
    }
]

print(f"Loaded {len(candidates)} candidate transients:")
for i, cand in enumerate(candidates, 1):
    print(f"  {i}. {cand['name']}: RA={cand['ra']:.1f}°, Dec={cand['dec']:.1f}°, z={cand['z']:.3f}")


Loaded 4 candidate transients:
  1. AT2024best: RA=120.2°, Dec=-30.1°, z=0.049
  2. AT2024okay: RA=121.5°, Dec=-29.0°, z=0.052
  3. AT2024poor: RA=125.0°, Dec=-35.0°, z=0.045
  4. AT2024bad: RA=180.0°, Dec=0.0°, z=0.100


### Rank Candidates

Use the `rank_candidates()` method to automatically rank all candidates by their association probability.


In [11]:
if assoc is not None and skymap_file:
    # Create association object for ranking (only need GW time)
    assoc_ranking = Association(
        gw_file=skymap_file,
        transient_info={"gw_time": 1234567880.0}
    )
    
    # Rank all candidates
    rankings = assoc_ranking.rank_candidates(candidates)
    
    print("=" * 80)
    print("CANDIDATE RANKINGS")
    print("=" * 80)
    print(f"{'Rank':<6} {'Name':<15} {'RA':<8} {'Dec':<8} {'z':<8} {'P(Assoc)':<12} {'Log Odds':<12} {'Decision'}")
    print("-" * 80)
    
    for i, r in enumerate(rankings, 1):
        cand = r['candidate']
        prob = r['probability']
        log_odds = r['log_odds']
        decision = "✓ ASSOC" if prob > 0.5 else "✗ UNLIKELY"
        
        print(f"{i:<6} {cand.name:<15} {cand.ra:<8.1f} {cand.dec:<8.1f} "
              f"{cand.z if cand.z else 'N/A':<8} {prob:<12.1%} {log_odds:<12.2f} {decision}")
    
    # Highlight top candidate
    best = rankings[0]
    print(f"\n{'=' * 80}")
    print(f"TOP CANDIDATE: {best['candidate'].name}")
    print(f"  P(Associated) = {best['probability']:.1%}")
    print(f"  Posterior Odds = {best['odds']:.3e}")
    print(f"  Log₁₀ Odds = {best['log_odds']:.2f}")
    print(f"{'=' * 80}")
else:
    print("⚠ Cannot rank candidates - association object not created")
    rankings = None


⚠ Cannot rank candidates - association object not created


### Visualize Rankings

Create a plot showing the candidate rankings.


In [12]:
if rankings is not None:
    try:
        # Create ranking plot
        ranking_file = "candidate_ranking.png"
        plot_candidate_ranking(rankings, ranking_file)
        print(f"✓ Ranking plot saved to: {ranking_file}")
        
        # Display the plot inline
        from IPython.display import Image, display
        if Path(ranking_file).exists():
            display(Image(ranking_file))
    except Exception as e:
        print(f"⚠ Could not create ranking plot: {e}")
        import traceback
        traceback.print_exc()
else:
    print("⚠ Cannot create plot - no rankings available")


⚠ Cannot create plot - no rankings available


## Advanced Examples

### Different EM Models

The framework supports different EM counterpart models, each with different temporal profiles:

- **kilonova**: Optical/NIR transients that peak ~1 day after merger
- **grb**: Gamma-ray bursts with prompt emission ~seconds after merger
- **afterglow**: GRB afterglows that peak ~1 day and decay over days/weeks

Let's compare how the same transient is evaluated under different models.


In [13]:
if assoc is not None and skymap_file:
    # Transient detected quickly after GW (2 seconds)
    fast_transient = {
        "ra": 120.5,
        "dec": -30.0,
        "z": 0.05,
        "time": 1234567882.0,  # 2 seconds after GW
        "gw_time": 1234567880.0
    }
    
    print("Transient detected 2 seconds after GW merger")
    print("Comparing different EM models:")
    print("-" * 70)
    print(f"{'Model':<12} {'I_t':<12} {'I_Ω':<12} {'I_DL':<12} {'P(Assoc)':<12}")
    print("-" * 70)
    
    models = ['grb', 'kilonova', 'afterglow']
    model_results = {}
    
    for model in models:
        assoc_model = Association(skymap_file, fast_transient.copy())
        results_model = assoc_model.compute_odds(em_model=model)
        model_results[model] = results_model
        
        print(f"{model:<12} {results_model['I_t']:<12.6e} "
              f"{results_model['I_omega']:<12.6e} {results_model['I_dl']:<12.6e} "
              f"{results_model['confidence']:<12.1%}")
    
    print("-" * 70)
    print("\nInterpretation:")
    print("  • GRB model: Best for prompt emission (seconds after merger)")
    print("  • Kilonova model: Best for optical/NIR transients (hours-days)")
    print("  • Afterglow model: Best for late-time emission (days-weeks)")
    print("\n  This rapid detection (2 seconds) suggests GRB/prompt emission!")
else:
    print("⚠ Cannot compare models - association object not created")


⚠ Cannot compare models - association object not created


### Custom Priors and Parameters

You can adjust the analysis parameters based on your science case:

- **Prior Odds**: Adjust based on GW source type
  - BNS (Binary Neutron Star): 1.0 (EM emission expected)
  - NSBH (Neutron Star-Black Hole): 0.1 (EM emission possible)
  - BBH (Binary Black Hole): 0.01 (EM emission unlikely)

- **Chance Coincidence Rate**: Expected rate of unrelated transients
- **H0 Uncertainty**: Hubble constant uncertainty (affects distance calculations)


In [14]:
if assoc is not None:
    # Same transient, different priors
    test_transient = {
        "ra": 120.5,
        "dec": -30.0,
        "z": 0.05,
        "time": 1234567891.0,
        "gw_time": 1234567880.0
    }
    
    print("Comparing different prior odds (same transient):")
    print("-" * 70)
    print(f"{'Source Type':<15} {'Prior Odds':<12} {'Posterior Odds':<18} {'P(Assoc)':<12}")
    print("-" * 70)
    
    source_types = {
        'BNS': 1.0,
        'NSBH': 0.1,
        'BBH': 0.01
    }
    
    for source_type, prior in source_types.items():
        assoc_prior = Association(skymap_file, test_transient.copy())
        results_prior = assoc_prior.compute_odds(prior_odds=prior)
        
        print(f"{source_type:<15} {prior:<12.2f} "
              f"{results_prior['posterior_odds']:<18.6e} "
              f"{results_prior['confidence']:<12.1%}")
    
    print("-" * 70)
    print("\nNote: Lower prior odds (e.g., BBH) require stronger evidence")
    print("      (higher Bayes factor) to reach the same posterior probability.")
else:
    print("⚠ Cannot test priors - association object not created")


⚠ Cannot test priors - association object not created


## Real-World Workflow

### Step-by-Step Guide for Analyzing Real GW Events

Here's a complete workflow for analyzing real GW events with EM transients:

#### Step 1: Get GW Skymap from GraceDB

1. Visit [GraceDB](https://gracedb.ligo.org/superevents/public/O4/)
2. Browse events or search for a specific event (e.g., `S250818k`)
3. Download the `bayestar.fits.gz` or `bilby.fits.gz` file
4. These are 3D skymaps with distance information (preferred)

**Example:**
```python
# Download skymap (manual or via API)
skymap_file = "S250818k_bayestar.fits.gz"
```

#### Step 2: Prepare EM Transient Data

Collect transient information from your observations:
- Position (RA, Dec) from astrometry
- Redshift (z, z_err) from spectroscopy
- Detection time (GPS or MJD)
- Magnitude and filter band (optional)

**Example:**
```python
transient_info = {
    "name": "AT2024abc",
    "ra": 192.42625,      # From your observations
    "dec": 34.82472,
    "z": 0.438,           # From spectroscopy
    "z_err": 0.005,
    "time": 1242442967.447,  # Detection time (GPS)
    "gw_time": 1242442965.0  # From GraceDB event page
}
```

#### Step 3: Run Association Analysis

```python
from gw_assoc import Association

assoc = Association(skymap_file, transient_info)
results = assoc.compute_odds(
    em_model='kilonova',
    prior_odds=1.0,  # Adjust for source type
    chance_coincidence_rate=1e-4
)

print(f"P(Associated) = {results['confidence']:.1%}")
```

#### Step 4: Rank Multiple Candidates

If you have multiple candidates:

```python
candidates = [
    {"name": "AT2024abc", "ra": 192.4, "dec": 34.8, "z": 0.438, ...},
    {"name": "AT2024def", "ra": 192.5, "dec": 34.9, "z": 0.440, ...},
    # ... more candidates
]

rankings = assoc.rank_candidates(candidates)
# Top candidate: rankings[0]
```

#### Step 5: Generate Plots for Publication

```python
assoc.plot_skymap("skymap_figure.png")
plot_association_summary(results, "summary_figure.png")
```

### Tips for Real Data

1. **GW Event Time**: Always get from GraceDB for accurate temporal calculations
2. **Redshift Quality**: Higher quality redshifts improve distance overlap
3. **3D Skymaps**: Prefer 3D skymaps (bayestar.fits.gz, bilby.fits.gz) over 2D
4. **EM Model Selection**: Choose based on detection time and wavelength
5. **Multiple Candidates**: Always rank to identify most likely counterpart


## Summary and Next Steps

### Key Takeaways

1. **Position Match (I_Ω) is Crucial**: Being in the high-probability region of the GW skymap matters most for association.

2. **Distance Agreement (I_DL) Helps Confirm**: When available, redshift information significantly improves association confidence.

3. **Time Delay (I_t) Depends on EM Model**: Choose the appropriate model (kilonova/GRB/afterglow) based on detection time and wavelength.

4. **Rank Multiple Candidates**: Always rank multiple candidates by posterior odds to identify the most likely counterpart.

5. **Adjust Priors for Source Type**: BNS events have higher prior odds for EM emission than BBH events.


Where:
- $I_Ω$ = Spatial overlap integral
- $I_{DL}$ = Distance overlap integral  
- $I_t$ = Temporal overlap integral
- $P_{\text{chance}}$ = Chance coincidence probability

### Next Steps

1. **Try with Real Data**: Download skymaps from [GraceDB](https://gracedb.ligo.org/superevents/)
2. **Explore Parameters**: Adjust priors, models, and coincidence rates for your science case
3. **Batch Processing**: Use the framework to analyze multiple events
4. **Custom Models**: Extend the temporal models for specific EM counterparts

### Documentation

- **Full Documentation**: See `README.md` for comprehensive documentation
- **API Reference**: Check docstrings in the source code
- **Examples**: See `working_example.py` and `examples.py` for more examples
- **Tests**: Run `python test_gw_assoc.py` to verify installation


**Happy analyzing! 🌌**
